In [1]:
import pandas as pd 
from functools import reduce
import string
# from spellchecker import SpellChecker
# spell = SpellChecker()
import nltk
porter = nltk.stem.PorterStemmer()
import numpy as np
import re
pd.options.mode.chained_assignment = None
from nltk.stem.wordnet import WordNetLemmatizer
lem = WordNetLemmatizer()
#nltk.download('wordnet')
import math

In [2]:
user = pd.read_csv("data/c_user.csv")
tip = pd.read_csv("data/c_tip.csv")
rev = pd.read_csv("data/c_review.csv")
business = pd.read_csv("data/cinema.csv")

In [3]:
def append(x,y):
    return x+y
def clean(x):
    return list(filter(lambda x: False if x in stop_words or x == '' else True, x))
def word_process(s):
    s = s.translate(str.maketrans('', '', string.punctuation))
    return lem.lemmatize(s, "v") 

In [4]:
stop_words = pd.read_table("data/stop_words.txt")
stop_words = list(stop_words["words"])

In [5]:
review = tip
review

,Unnamed: 0,user_id,business_id,text,date,compliment_count
0,67,_RvEDmpbQZulEdAV590QKg,xQLs0A2dzgXgJq6K2U_UUA,Annoying they don't offer baby and me movies t...,2012-01-09 19:46:53,0
1,116,udpfGxriuoBdWRBdplJl-w,jobP3ywRd3QNZ_GCoPG2DQ,Finally watching transformers 3d imax.,2011-07-13 02:37:57,0
2,126,93z51tzHUp4ycmC45puYRg,h6Y1-KfN2XBkcrV85CS8yg,Seats were good. Food was ok. Decent drinks. B...,2014-08-29 15:14:09,0
3,209,2i4n1dxsxQL2C2eRTJwdmQ,-ipc5f0XiXwH3hbNnro54Q,21 jumpstreet....Channing Tatum,2012-03-24 04:57:49,0
4,407,t_uV5QZeKuuiTEEMsGPtQg,2fqHO2vFbcwVV27c_Irm8Q,The best movie theater!,2016-12-06 18:53:16,0
...,...,...,...,...,...,...
11351,1222070,K6QG9eh8hXbW7oIypWQ-mQ,A0FLFsqkGK59l2WPhi93mg,"""Braving"" the possible storms",2012-07-02 00:16:24,0
11352,1222087,FO2MgN2nPNcjFAcWYP7CnQ,WgD4husYU3KNF1nOlg2MEg,"Tres bon cinema, jai habiter dans le coin pend...",2016-02-29 18:35:38,0
11353,1222360,CZFicRCrK_6v4jE4mTF7gw,RtXj1StQUbCMEmxRR10Bqg,Zero Dark Thirty! I'm proud to be a American.....,2013-01-13 05:34:40,0
11354,1222468,5MfsMof-_ddB6MqdoyMXDQ,VkOyrR6CDfhZ7hh4yOvhiw,About to watch Horrible Bosses... OMG The movi...,2011-08-02 01:06:00,0


In [6]:
word_init = list(map(lambda x: x.replace('no ', 'no').replace('never ', 'never').replace('not ', 'not').replace("n't ", ' not').replace('nt ', ' not').replace("n' ", " not").replace("should ","should").replace("would ", "would").lower().replace('\n', ' ').split(' '), list(review["text"])))

In [7]:
# sample = list(np.random.choice(len(word_init), size=100, replace=False))
# word = list(map(lambda x: list(map(word_process, x)), list(pd.Series(word_init)[sample])))
word = list(map(lambda x: list(map(word_process, x)), word_init))

In [8]:
word_clean = list(map(clean, word))

In [9]:
word_dict = {}
word_dict_no_weight = {}
review_list = review
review_list["text"] = pd.Series(word_clean)
for i in list(review.index.values):
    data = review.loc[i]
    if type(data["text"]) == float:
        continue
    count = 1+data["compliment_count"]
    time = (np.array('2019-01-01', dtype=np.datetime64)-np.array(review_list.loc[i]["date"].split(" ")[0], dtype=np.datetime64)).astype(dtype=np.float32)
    time_weight = 3-(time/365) if (3-(time/365))  > 0 else 0
    for w in review_list.loc[i]["text"]:
        if not w in word_dict:
            word_dict[w] = 0
            word_dict_no_weight[w] = 0
        word_dict[w] += count*(0.8+time_weight)
        word_dict_no_weight[w] += 1
word_dict

{'annoy': 16.975342465753428,
 'notoffer': 1.6,
 'baby': 57.14794520547938,
 'movies': 513.6712328767151,
 'week': 54.479452054794464,
 'hood': 2.4000000000000004,
 'moms': 4.0,
 'finally': 84.12328767123276,
 'watch': 533.8712328767155,
 'transformers': 39.19999999999999,
 '3d': 273.29315068493315,
 'imax': 154.85205479452057,
 'seat': 1023.0575342465652,
 'good': 493.17808219178386,
 'food': 323.5342465753436,
 'ok': 39.24657534246574,
 'dece': 34.709589041095896,
 'notdrinks': 0.8,
 'better': 154.49863013698643,
 'regular': 36.3972602739726,
 'movie': 1290.4849315068266,
 '21': 58.71506849315061,
 'jumpstreetchanning': 0.8,
 'tatum': 1.6,
 'best': 326.7589041095902,
 'theater': 741.7260273972557,
 'nothave': 53.731506849315025,
 'problems': 10.317808219178083,
 'omg': 14.950684931506853,
 'insidious': 9.6,
 'happy': 51.038356164383515,
 'hour': 49.73150684931504,
 'cinebarre': 3.2,
 'sherlock': 22.40000000000001,
 'holmes': 19.200000000000006,
 '2': 277.95890410959026,
 'techmunch':

In [10]:
# count_filter = pd.Series(word_dict) >= sorted(list(word_dict.values()))[-50]
# count_filter = count_filter.reset_index()[0]
# pd.Series(list(word_dict.keys()))[count_filter]
def word_dict_fx(key):
    return word_dict[key]
def word_dict_no_weight_fx(key):
    return word_dict_no_weight[key]

In [11]:
hp_word = sorted(list(word_dict.keys()), key=word_dict_fx, reverse = True)
hp_word_count = pd.Series(word_dict)[hp_word]

hp_word_no_weight = sorted(list(word_dict_no_weight.keys()), key=word_dict_no_weight_fx, reverse = True)
hp_word_count_no_weight = pd.Series(word_dict_no_weight)[hp_word_no_weight]

In [12]:
import csv
w = csv.writer(open("word_count/tip_count_weight" + ".csv", "w"))
for key, val in hp_word_count.items():
    w.writerow([key, val])
    
w = csv.writer(open("word_count/tip_count" + ".csv", "w"))
for key, val in hp_word_count_no_weight.items():
    w.writerow([key, val])